<a href="https://colab.research.google.com/github/RitikaHiremath/DataScience/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd
import os

def load_and_label_data(base_path, label, max_files=None):
    combined_100KHzdata = []
    combined_2000KHzdata = []

    # Initialize a counter
    file_counter = 0

    # Iterate over each timestamped folder
    for timestamp_folder in os.listdir(base_path):
        if max_files and file_counter >= max_files:
            break

        timestamp_folder_path = os.path.join(base_path, timestamp_folder, "raw")

        # Extract the full timestamp from the folder name and convert to datetime format
        timestamp = timestamp_folder.split('_')[0] + '_' + timestamp_folder.split('_')[1]
        timestamp = pd.to_datetime(timestamp, format='%Y.%m.%d_%H.%M.%S')

        # Load the 2000KHz data
        df_2000KHz = pd.read_parquet(os.path.join(timestamp_folder_path, "Sampling2000KHz_AEKi-0.parquet"))
        mean_2000KHz = df_2000KHz.mean().to_frame().T
        mean_2000KHz['timestamp'] = timestamp
        mean_2000KHz['label'] = label

        # Load the 100KHz data
        df_100KHz = pd.read_parquet(os.path.join(timestamp_folder_path, "Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet"))
        mean_100KHz = df_100KHz.mean().to_frame().T
        mean_100KHz['timestamp'] = timestamp
        mean_100KHz['label'] = label

        # Append the mean data to the combined lists
        combined_100KHzdata.append(mean_100KHz)
        combined_2000KHzdata.append(mean_2000KHz)

        # Increment the counter
        file_counter += 1

    # Combine all the mean data into a single dataframe
    final_combined_100KHzdata = pd.concat(combined_100KHzdata, ignore_index=True)
    final_combined_2000KHzdata = pd.concat(combined_2000KHzdata, ignore_index=True)

    return final_combined_100KHzdata, final_combined_2000KHzdata

# Define the paths to the OK and NOK data directories
ok_data_path = '/content/gdrive/MyDrive/Data/OK_Measurements'
nok_data_path = '/content/gdrive/MyDrive/Data/NOK_Measurements'

# Load OK and NOK data
ok_100KHzdata, ok_2000KHzdata = load_and_label_data(ok_data_path, label=0)
nok_100KHzdata, nok_2000KHzdata = load_and_label_data(nok_data_path, label=1)

# Combine OK and NOK data
all_100KHzdata = pd.concat([ok_100KHzdata, nok_100KHzdata], ignore_index=True)
all_2000KHzdata = pd.concat([ok_2000KHzdata, nok_2000KHzdata], ignore_index=True)

# Print the first few rows of the combined data for inspection
print("Combined 100KHz Data Sample (Mean):")
print(all_100KHzdata.head())
print("\nCombined 2000KHz Data Sample (Mean):")
print(all_2000KHzdata.head())

Combined 100KHz Data Sample (Mean):
   Irms_Grinding_rate100000_clipping0_batch0  \
0                                   0.085166   
1                                   0.085681   
2                                   0.085834   
3                                   0.085607   
4                                   0.085260   

   Grinding spindle current L1_rate100000_clipping0_batch0  \
0                                          -0.000076         
1                                          -0.000062         
2                                          -0.000078         
3                                          -0.000086         
4                                          -0.000060         

   Grinding spindle current L2_rate100000_clipping0_batch0  \
0                                           0.000064         
1                                           0.000100         
2                                           0.000069         
3                                           0.000083  

In [4]:
from sklearn.preprocessing import StandardScaler

# Separate features and labels for 100KHz data
features_100KHz = all_100KHzdata.drop(columns=['timestamp', 'label'])  # Exclude timestamp and label
timestamps_100KHz = all_100KHzdata['timestamp']
labels_100KHz = all_100KHzdata['label']

# Normalize features for 100KHz data
scaler_100KHz = StandardScaler()
normalized_features_100KHz = scaler_100KHz.fit_transform(features_100KHz)

# Combine normalized features with timestamps and labels
normalized_100KHzdata = pd.DataFrame(normalized_features_100KHz, columns=features_100KHz.columns)
normalized_100KHzdata.insert(0, 'timestamp', timestamps_100KHz)  # Add timestamp column back
normalized_100KHzdata['label'] = labels_100KHz.values  # Add label column back

# Separate features and labels for 2000KHz data
features_2000KHz = all_2000KHzdata.drop(columns=['timestamp', 'label'])  # Exclude timestamp and label
timestamps_2000KHz = all_2000KHzdata['timestamp']
labels_2000KHz = all_2000KHzdata['label']

# Normalize features for 2000KHz data
scaler_2000KHz = StandardScaler()
normalized_features_2000KHz = scaler_2000KHz.fit_transform(features_2000KHz)

# Combine normalized features with timestamps and labels
normalized_2000KHzdata = pd.DataFrame(normalized_features_2000KHz, columns=features_2000KHz.columns)
normalized_2000KHzdata.insert(0, 'timestamp', timestamps_2000KHz)  # Add timestamp column back
normalized_2000KHzdata['label'] = labels_2000KHz.values  # Add label column back

print("Normalized 100KHz Data Sample:")
print(normalized_100KHzdata.head())
print(len(normalized_100KHzdata))
print("\nNormalized 2000KHz Data Sample:")
print(normalized_2000KHzdata.head())
print(len(normalized_2000KHzdata))

Normalized 100KHz Data Sample:
            timestamp  Irms_Grinding_rate100000_clipping0_batch0  \
0 2024-02-14 22:00:10                                  -1.213932   
1 2024-02-14 22:04:13                                  -1.059871   
2 2024-02-14 22:05:15                                  -1.014171   
3 2024-02-14 22:03:43                                  -1.082180   
4 2024-02-14 22:01:11                                  -1.185769   

   Grinding spindle current L1_rate100000_clipping0_batch0  \
0                                           0.410776         
1                                           1.275351         
2                                           0.286297         
3                                          -0.208334         
4                                           1.423043         

   Grinding spindle current L2_rate100000_clipping0_batch0  \
0                                          -0.126537         
1                                           1.918509         
2

In [5]:
import pandas as pd

# Concatenate the 100KHz and 2000KHz data
normalized_100KHzdata = normalized_100KHzdata.set_index('timestamp')
normalized_2000KHzdata = normalized_2000KHzdata.set_index('timestamp')

# Concatenate along columns
combined_data = pd.concat([normalized_100KHzdata, normalized_2000KHzdata], axis=1, join='inner').reset_index()

# Remove duplicate 'label' columns and keep the first one
combined_data = combined_data.loc[:, ~combined_data.columns.duplicated()]

# Ensure the label column is at the end
label = combined_data.pop('label')
combined_data['label'] = label

print("Combined Data Sample:")
print(combined_data.head())
print(len(combined_data))

Combined Data Sample:
            timestamp  Irms_Grinding_rate100000_clipping0_batch0  \
0 2024-02-14 22:00:10                                  -1.213932   
1 2024-02-14 22:04:13                                  -1.059871   
2 2024-02-14 22:05:15                                  -1.014171   
3 2024-02-14 22:03:43                                  -1.082180   
4 2024-02-14 22:01:11                                  -1.185769   

   Grinding spindle current L1_rate100000_clipping0_batch0  \
0                                           0.410776         
1                                           1.275351         
2                                           0.286297         
3                                          -0.208334         
4                                           1.423043         

   Grinding spindle current L2_rate100000_clipping0_batch0  \
0                                          -0.126537         
1                                           1.918509         
2         

In [40]:
import tensorflow as tf
from tensorflow.keras.losses import Loss

class CustomLoss(Loss):
    def __init__(self, name="custom_loss"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        main_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        auxiliary_loss = self.calculate_auxiliary_loss(y_true, y_pred)
        total_loss = main_loss + auxiliary_loss
        return total_loss

    def calculate_auxiliary_loss(self, y_true, y_pred):
        mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)

In [ ]:
!pip install tensorflow_decision_forests

In [23]:
# https://www.tensorflow.org/decision_forests/tutorials/beginner_colab

import tensorflow as tf
import tensorflow_decision_forests as tfdf


# without 'timestamp'
ndf = combined_data.loc[:, combined_data.columns != 'timestamp']

# rename?

#  tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(ndf, label = 'label')

#  model = tfdf.keras.RandomForestModel()
#  model.fit(tf_dataset)

#  print(model.summary())



Use /tmp/tmphwko7i1k as temporary training directory
Reading training dataset...


Training dataset read in 0:00:00.289120. Found 58 examples.
Training model...
Model trained in 0:00:00.045389
Compiling model...
Model compiled.


Model: "random_forest_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (5):
	AEKi_rate2000000_clipping0_batch0
	Grinding_spindle_current_L1_rate100000_clipping0_batch0
	Grinding_spindle_current_L2_rate100000_clipping0_batch0
	Grinding_spindle_current_L3_rate100000_clipping0_batch0
	Irms_Grinding_rate100000_clipping0_batch0

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "Irms_Grinding_rate100000_clipping0_batch0"  0.675676 ################
    2.         "AEKi_rate2000000_clipping0_batch0"  0.657895 

Variable Importance: NUM_AS_ROOT:
    1. "Irms_Grinding_rate100000_clipping0_batch0" 156.000000 ################
    2.         "AEKi_rate2000000_clip

In [27]:

ndf.columns
ndf.head(3)



,Irms_Grinding_rate100000_clipping0_batch0,Grinding spindle current L1_rate100000_clipping0_batch0,Grinding spindle current L2_rate100000_clipping0_batch0,Grinding spindle current L3_rate100000_clipping0_batch0,AEKi_rate2000000_clipping0_batch0,label
0,-1.213932,0.410776,-0.126537,-0.436171,-1.575957,0
1,-1.059871,1.275351,1.918509,-0.403926,-0.388929,0
2,-1.014171,0.286297,0.143756,0.565636,-0.294156,0


In [28]:
# encode the categorical labels as integers
label = 'label'

classes = ndf[label].unique().tolist()
print(f"Label clsses: {classes}")

ndf[label] = ndf[label].map(classes.index)

Label clsses: [0, 1]


<ipython-input-28-074a6774b075>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf[label] = ndf[label].map(classes.index)


In [29]:
# split the dataset into training and testing
import numpy as np
def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(ndf)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

44 examples in training, 14 examples for testing.


In [34]:
# convert the pandas dataframe into tensorflow datasets

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label = label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label = label)


In [ ]:
# specify the model
rf_model = tfdf.keras.RandomForestModel(verbose = 2)

# train the model
rf_model.fit(train_ds)

In [41]:
# evaluate the model
rf_model.compile(metrics = ["accuracy"])
# rf_model.compile(optimizer = 'adam', loss = CustomLoss(), metrics = ['accuracy'])
evaluation = rf_model.evaluate(test_ds, return_dict = True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 0s 270ms/step - loss: 0.0000e+00 - accuracy: 1.0000

loss: 0.0000
accuracy: 1.0000


In [ ]:
# from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
# from sklearn.decomposition import PCA
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score

# import tensorflow as tf
# import tensorflow_decision_forests as tfdf

# combined_data.columns
# without 'timestamp'
#features = ['Irms_Grinding_rate100000_clipping0_batch0',
#      'Grinding spindle current L1_rate100000_clipping0_batch0',
#       'Grinding spindle current L2_rate100000_clipping0_batch0',
#       'Grinding spindle current L3_rate100000_clipping0_batch0',
#       'AEKi_rate2000000_clipping0_batch0']
#target = ['label']

# X = combined_data[features]
# y = combined_data[target]

# splitting train and test data
# X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .3, random_state = 42)

# PCA ?
# pca = PCA().fit(X_tr)
# X_tr = pca.transform(X_tr)
# X_ts = pca.transform(X_ts)

# Random Forest
#train_dataset = tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
# train_dataset = train_dataset.batch(30)

# tfrf = tfdf.keras.RandomForestModel(tfdf.keras.Task.CLASSIFICATION)
# tfrf.compile(optimizer = 'adam', loss = '', metrics = ['accuracy'])
# tfrf.fit(train_dataset)

# print(tfrf.summary())


